In [1]:
import numpy as np
from PIL import Image

from pycoral.adapters import common
from pycoral.adapters import segment
from pycoral.utils.edgetpu import make_interpreter
 
interpreter = make_interpreter('deeplabv3_mnv2_pascal_quant_edgetpu.tflite', device=':0')
interpreter.allocate_tensors()
width, height = common.input_size(interpreter)



In [2]:
from io import BytesIO
from pycoral.adapters import common
from pycoral.adapters import segment
from pycoral.utils.edgetpu import make_interpreter
from tpu_seg import label_to_color_image


def run_visualization_img(im, interpreter):
    """Inferences DeepLab model and visualizes result."""
    img=Image.fromarray(np.uint8(im))
    resized_img, _ = common.set_resized_input(
        interpreter, img.size, lambda size: img.resize(size, Image.ANTIALIAS))

    interpreter.invoke()
    result = segment.get_output(interpreter)
    if len(result.shape) == 3:
    # result = np.argmax(result, axis=-1)
        result = np.argmax(result, axis=-1)

    new_width, new_height = resized_img.size
    result = result[:new_height, :new_width]
    mask_img = Image.fromarray(label_to_color_image(result).astype(np.uint8))

  #pp=vis_segmentation(resized_im, seg_map)
    return mask_img


#from PIL import Image
#im = Image.open('images.jpeg')
#mask_img=run_visualization_img(im, interpreter)

In [3]:

from jetbot import Camera
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, usb=0)
from jetbot import bgr8_to_jpeg
import traitlets
import ipywidgets

#image_w = ipywidgets.Image()
image_w = widgets.Image(format='jpeg', width=224, height=224)

traitlets.dlink((camera, 'value'), (image_w, 'value'), transform=bgr8_to_jpeg)


In [4]:
seg_image = ipywidgets.Image(format='jpeg', width=224, height=224)
display(widgets.VBox([
    widgets.HBox([image_w, seg_image]),
]))

In [5]:
import io

def execute(change):
    image = change['new']
    mask =run_visualization_img(camera.value, interpreter)

    buf = io.BytesIO()
    mask.save(buf, format='JPEG')
    seg_image.value = buf.getvalue()

    #mask = 1.0 * (output.argmax(0) == 15)
    #m = mask[:, :, None]
    #print(m.shape)
    #print(image.shape)
 #   seg_image.value = bgr8_to_jpeg(mask[:, :, None] * image)
   # seg_image.value = bgr8_to_jpeg(mask)
    


In [6]:
mask = execute({'new': camera.value})
camera.observe(execute, names='value')

In [ ]:
camera.stop()
camera.unobserve(execute, names='value')
